In [1]:
from pyspark.sql import SparkSession,Row,DataFrame
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
import pyspark.sql.functions as F
import os.path as path
import traceback


builder = SparkSession \
    .builder \
    .appName("emp") \
    .master("local[4]") \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.warehouse.dir", "/Users/eduardoalberto/LoadFile/output/") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.5.3/jars/mysql-connector-j-9.1.0.jar" ) \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

25/05/08 22:50:34 WARN Utils: Your hostname, MacBook-Pro-de-Eduardo.local resolves to a loopback address: 127.0.0.1; using 192.168.15.9 instead (on interface en0)
25/05/08 22:50:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/eduardoalberto/opt/spark-3.5.4/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/eduardoalberto/.ivy2/cache
The jars for the packages stored in: /Users/eduardoalberto/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-539a7e76-98cd-4c8a-92c8-f6bb6b4aadb1;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 240ms :: artifacts dl 14ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default

PySpark Version :3.5.4
PySpark Version :3.5.4


In [ ]:
df = spark.read.csv("/Users/eduardoalberto/LoadFile/input/kc_house_data.csv", header=True,inferSchema=True)\
                .withColumn("id_number", F.monotonically_increasing_id())
df.show(truncate=False)

### GERA DELTATABLE

In [ ]:
path_delta = "/Users/eduardoalberto/LoadFile/output/emp"
df = df.withColumn("dt_ref_carga", F.current_date())



df.write.format("delta")\
        .option("overwriteSchema", "true")\
        .option("path",path_delta)\
        .partitionBy("dt_ref_carga")\
        .mode("overwrite")\
        .saveAsTable("default.tb_emp")


# df.write.format("delta")\
#         .option("overwriteSchema", "true")\
#         .partitionBy("dt_ref_carga")\
#         .mode("overwrite")\
#         .saveAsTable("default.tb_emp")\



In [ ]:
spark.sql("describe history tb_emp").show()


### CONSULTA DELTATABLE

In [ ]:
# spark.read.format("delta").load(path_delta).toPandas()

### SCRIPT FULL

In [2]:
def ingest(spark):

    CONF = {'inptFile': '/Users/eduardoalberto/LoadFile/input/kc_house_data.csv',
            'path_delta': '/Users/eduardoalberto/LoadFile/output/emp',
            'JDBC':  "jdbc:mysql://localhost:3306/myDbUser",
            'TABLE': "myDbUser.emp",
            'DB_USER': 'root',
            'DB_PASSWORD': 'mysql'
        }
    try:
        if path.isfile(CONF["inptFile"]):
            
            kc_house_dt = spark.read.csv(CONF["inptFile"], header=True,inferSchema=True)\
                                    .withColumn("id_number", F.monotonically_increasing_id())\
                                    .withColumn("dt_ref_carga", F.current_date())
            
            dfs = kc_house_dt.select(   F.coalesce(F.col('id'),F.lit(0)).alias('id'),
                                        F.coalesce(F.col('dt_date'),F.lit(0)).alias('dt_date'),
                                        F.coalesce(F.col('price'),F.lit(0)).alias('price'),
                                        F.coalesce(F.col('bedrooms'),F.lit(0)).alias('bedrooms'),
                                        F.coalesce(F.col('bathrooms'),F.lit(0)).alias('bathrooms'),
                                        F.coalesce(F.col('sqft_living'),F.lit(0)).alias('sqft_living'),
                                        F.coalesce(F.col('sqft_lot'),F.lit(0)).alias('sqft_lot'),
                                        F.coalesce(F.col('floors'),F.lit(0)).alias('floors'),
                                        F.coalesce(F.col('waterfront'),F.lit(0)).alias('waterfront'),
                                        F.coalesce(F.col('view'),F.lit(0)).alias('view'),
                                        F.coalesce(F.col('condition'),F.lit(0)).alias('condition'),
                                        F.coalesce(F.col('grade'),F.lit(0)).alias('grade'),
                                        F.coalesce(F.col('sqft_above'),F.lit(0)).alias('sqft_above'),
                                        F.coalesce(F.col('sqft_basement'),F.lit(0)).alias('sqft_basement'),
                                        F.coalesce(F.col('yr_built'),F.lit(0)).alias('yr_built'),
                                        F.coalesce(F.col('yr_renovated'),F.lit(0)).alias('yr_renovated'),
                                        F.coalesce(F.col('zipcode'),F.lit(0)).alias('zipcode'),
                                        F.coalesce(F.col('lat'),F.lit(0)).alias('lat'),
                                        F.coalesce(F.col('long'),F.lit(0)).alias('long'),
                                        F.coalesce(F.col('sqft_living15'),F.lit(0)).alias('sqft_living15'),
                                        F.coalesce(F.col('sqft_lot15'),F.lit(0)).alias('sqft_lot15'),
                                        F.coalesce(F.col('id_number'),F.lit(0)).alias('id_number'),
                                        F.coalesce(F.col('dt_ref_carga'),F.lit('1900-01-01')).alias('dt_ref_carga'))



            dfs.write.format("delta")\
                    .option("overwriteSchema", "true")\
                    .option("path",CONF["path_delta"])\
                    .partitionBy("dt_ref_carga")\
                    .mode("overwrite")\
                    .saveAsTable("default.tb_emp")
            # mysql
            dfs.write.mode("overwrite").jdbc(CONF["JDBC"], CONF["TABLE"],properties={"user": CONF["DB_USER"],"password": CONF["DB_PASSWORD"]})

        else:
            print("arquivo não existe!")

    except Exception as e:
        print(f"Ocorreu o seguinte erro: {e}")
        traceback.print_exc()
    return dfs

df = ingest(spark)

df.count()






21613